In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from adversarial_dataset import AdversarialDataset
import os
import matplotlib.pyplot as plt
from torchvision import models
from tqdm.notebook import tqdm
import numpy as np


In [2]:
class TensorToDevice(object):
    def __init__(self, device):
        self.device = device
        
    def __call__(self, image):
        image = image.to(self.device)
        return image

In [3]:
def show_image(datarow):
    permuted = torch.permute(datarow[0].cpu(), (1, 2, 0))
    plt.title(datarow[1])
    plt.imshow(permuted)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = transforms.Compose([
    transforms.ToTensor(),
    TensorToDevice(device),
    transforms.Normalize(mean=mean, std=std)
])

adv_transform = transforms.Compose([
    transforms.ToTensor(),
    TensorToDevice(device)
])

In [6]:
dataset = AdversarialDataset("images.csv", "categories.csv", "images", ".png", x_transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

adv_dataset = AdversarialDataset("images.csv", "categories.csv", "adversarial_images_gpu", ".npy", x_transform=transform)
adv_dataloader = DataLoader(adv_dataset, batch_size=32, shuffle=True)

model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model = model.to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

<h3>Training</h3>

In [7]:
n_epochs = 0
running_loss = 0
n_correct = 0

for epoch in range(n_epochs):
    for x_batch, y_batch in tqdm(dataloader):
        x_batch = x_batch.float()
        x_batch = x_batch.to(device)
        y_batch = y_batch.type(torch.LongTensor)
        y_batch = y_batch.to(device)
    
        output, _ = model(x_batch)
    
        loss = loss_function(output, y_batch)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() + x_batch.size(0)
    
        _, preds = torch.max(output, dim=1)
        n_correct += torch.sum(preds == y_batch.data)

<h3>Testing</h3>

In [8]:
for loader in [dataloader, adv_dataloader]:
    n_correct = 0
    for x_batch, y_batch, _, _ in tqdm(loader):
        y_batch = y_batch.to(device)
        output, _ = model(x_batch)
    
        _, preds = torch.max(output, dim=1)
        n_correct += torch.sum(preds == y_batch)
        
    print(n_correct/len(dataset))

# print(n_correct/len(dataset))

  0%|          | 0/32 [00:00<?, ?it/s]

tensor(0.8800, device='cuda:0')


  0%|          | 0/32 [00:00<?, ?it/s]

tensor(0.8010, device='cuda:0')


<h3> Generate C&W images </h3>

In [9]:
import cw_impl.cw as cw
import math
from torchvision.utils import save_image 

def generate_cw_samples(model, dataloader, save_dir, save_to_disk=True):
    n_original_correct, n_adv_correct = 0, 0
    
    inputs_box = (min((0 - m) / s for m, s in zip(mean, std)), max((1 - m) / s for m, s in zip(mean, std)))
    
    adversary = cw.L2Adversary(targeted=False, 
                               confidence=0.0,
                               search_steps=5,
                               abort_early=True,
                               box=inputs_box)
    
    model.eval()
    for i_batch, (inputs, labels, _, input_ids) in enumerate(dataloader):
        print(f"Batch {i_batch+1}/{math.ceil(1000/32)}")
        labels = labels.to(device)
        adversarial_examples = adversary(model, inputs, labels, to_numpy=False)
        adversarial_examples = adversarial_examples.to(device)
        
        adv_output = model(adversarial_examples)
        original_output = model(inputs)
        
        _, adv_preds = torch.max(adv_output, dim=1)
        n_adv_correct += torch.sum(adv_preds == labels)
        
        _, preds = torch.max(original_output, dim=1)
        n_original_correct += torch.sum(preds == labels)
        
        
        print(f"Total original correct: {n_original_correct} at batch {i_batch+1}")
        print(f"Total adversarial correct: {n_adv_correct} at batch {i_batch+1}")
        
        if save_to_disk:
            for i in range(adversarial_examples.shape[0]):
                save_image(adversarial_examples[i], os.path.join(save_dir, input_ids[i] + ".png"))
        
generate_cw_samples(model, dataloader, "test")
        

Batch 1/32
Total original correct: 29 at batch 1
Total adversarial correct: 0 at batch 1
Batch 2/32
Total original correct: 54 at batch 2
Total adversarial correct: 0 at batch 2
Batch 3/32
Total original correct: 81 at batch 3
Total adversarial correct: 0 at batch 3
Batch 4/32
Total original correct: 108 at batch 4
Total adversarial correct: 1 at batch 4
Batch 5/32
Total original correct: 133 at batch 5
Total adversarial correct: 1 at batch 5
Batch 6/32
Total original correct: 163 at batch 6
Total adversarial correct: 1 at batch 6
Batch 7/32
Total original correct: 190 at batch 7
Total adversarial correct: 1 at batch 7
Batch 8/32
Total original correct: 213 at batch 8
Total adversarial correct: 1 at batch 8
Batch 9/32
Total original correct: 241 at batch 9
Total adversarial correct: 1 at batch 9
Batch 10/32
Total original correct: 267 at batch 10
Total adversarial correct: 1 at batch 10
Batch 11/32
Total original correct: 295 at batch 11
Total adversarial correct: 1 at batch 11
Batch 1

In [10]:
import cw_impl.cw as cw

inputs_box = (min((0 - m) / s for m, s in zip(mean, std)), max((1 - m) / s for m, s in zip(mean, std)))
    
adversary = cw.L2Adversary(targeted=False, 
                           confidence=0.0,
                           search_steps=5,
                           abort_early=True,
                           box=inputs_box)

model.eval()

inputs, labels, targets, ids = next(iter(dataloader))

adversarial_examples = adversary(model, inputs, labels, to_numpy=True)
